In [1]:
import pandas as pd
import json

The script involves transforming and merging datasets into structured JSON files that match the predefined schemas for MongoDB integration. First, the games.csv and games_metadata.json files are loaded and merged on the app_id field to combine game attributes with metadata like descriptions and tags. The merged dataset is then transformed into a hierarchical JSON structure that includes fields like platforms, pricing, and tags. This structure ensures compatibility with MongoDB's document-oriented design. Similarly, the users.csv and recommendations.csv datasets are loaded and converted to JSON files. All transformed data is saved as structured JSON files (games.json, users.json, recommendations.json), making them ready for insertion into MongoDB.

In [8]:
# read the games csv and the games metadata
games = pd.read_csv('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/games.csv')
with open('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/games_metadata.json', 'r') as f:
    games_metadata = [json.loads(line) for line in f]

# Convert games_metadata into a DataFrame
games_metadata_df = pd.DataFrame(games_metadata)

# Merge games.csv and games_metadata.json on 'app_id'
merged_games = pd.merge(games, games_metadata_df, on='app_id', how='left')

# Transform merged_games to match the schema
def transform_games_schema(row):
    return {
        "app_id": row["app_id"],
        "title": row["title"],
        "description": row.get("description", ""),
        "tags": row.get("tags", []),
        "date_release": row["date_release"],
        "platforms": {
            "win": row["win"],
            "mac": row["mac"],
            "linux": row["linux"]
        },
        "rating": row["rating"],
        "positive_ratio": row["positive_ratio"],
        "user_reviews": row["user_reviews"],
        "price": {
            "final": row["price_final"],
            "original": row["price_original"],
            "discount": row["discount"]
        },
        "steam_deck": row["steam_deck"]
    }

# Transform each row into the correct structure
games_json = merged_games.apply(transform_games_schema, axis=1).tolist()

# Save the transformed JSON data
with open('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/games.json', 'w') as f:
    json.dump(games_json, f, indent=4)

In [12]:
# Load users.csv
users = pd.read_csv('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/users.csv')

# Convert all columns to Python-native types
users = users.astype('object')

# Transform users to match the schema
def transform_users_schema(row):
    return {
        "user_id": row["user_id"],
        "products": row["products"],
        "reviews": row["reviews"]
    }

# Transform users DataFrame to match the schema
users_json = users.apply(transform_users_schema, axis=1).tolist()

# Save the transformed JSON data to a file
with open('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/users.json', 'w') as f:
    json.dump(users_json, f, indent=4)


In [ ]:
# Load recommendations.csv
recommendations = pd.read_csv('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/recommendations.csv')

# Convert all columns to Python-native types
recommendations['app_id'] = recommendations['app_id'].astype(int)
recommendations['helpful'] = recommendations['helpful'].astype(int)
recommendations['funny'] = recommendations['funny'].astype(int)
recommendations['hours'] = recommendations['hours'].astype(float)
recommendations['user_id'] = recommendations['user_id'].astype(int)
recommendations['review_id'] = recommendations['review_id'].astype(int)

def transform_recommendations_schema(row):
    return {
        "review_id": row["review_id"],
        "app_id": row["app_id"],
        "user_id": row["user_id"],
        "helpful": row["helpful"],
        "funny": row["funny"],
        "date": row["date"],
        "is_recommended": row["is_recommended"],
        "hours_played": row["hours"]
    }

# Transform each row into the correct structure
recommendations_json = recommendations.apply(transform_recommendations_schema, axis=1).tolist()

# Save the transformed JSON data to a file
with open('/Users/chenyumiao/Desktop/ds5760/FinalProject/game_data/recommendations.json', 'w') as f:
    json.dump(recommendations_json, f, indent=4)